In [1]:
import torch
from torchvision import transforms, datasets


In [2]:
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize(256),
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406],[.229, 0.224, 0.225])
    ])
}


In [3]:
image_datasets = {
    'train': datasets.ImageFolder("../input/train", data_transforms['train']),
    'val': datasets.ImageFolder("../input/val", data_transforms['val'])
}


In [4]:
image_datasets

{'train': Dataset ImageFolder
     Number of datapoints: 188
     Root location: ../input/train
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=PIL.Image.BILINEAR)
                RandomHorizontalFlip(p=0.5)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            ),
 'val': Dataset ImageFolder
     Number of datapoints: 62
     Root location: ../input/val
     StandardTransform
 Transform: Compose(
                Resize(size=256, interpolation=PIL.Image.BILINEAR)
                ToTensor()
                Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
            )}

In [5]:
image_dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=4, shuffle=True, num_workers=0, drop_last=True),
    'val': torch.utils.data.DataLoader(image_datasets['val'], batch_size=4, shuffle=False, num_workers=0, drop_last=True),
    }


In [6]:
image_dataloaders

{'train': <torch.utils.data.dataloader.DataLoader at 0x7f605f708e10>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x7f605f708ed0>}

In [7]:
image_datasets['train']

Dataset ImageFolder
    Number of datapoints: 188
    Root location: ../input/train
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=PIL.Image.BILINEAR)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [8]:
for i, (inputs, labels) in enumerate(image_dataloaders['train']):
    print(inputs)
    print(labels)
    if i == 0:
        break

tensor([[[[0.8276, 0.8961, 0.8447,  ..., 1.1015, 1.1015, 1.1015],
          [0.8276, 0.8789, 0.8447,  ..., 1.1015, 1.1015, 1.1015],
          [0.8104, 0.8618, 0.8447,  ..., 1.0844, 1.0844, 1.0844],
          ...,
          [0.3138, 0.3138, 0.3309,  ..., 0.6734, 0.6734, 0.6734],
          [0.3138, 0.3309, 0.3309,  ..., 0.6734, 0.6734, 0.6734],
          [0.3309, 0.3309, 0.3309,  ..., 0.6734, 0.6734, 0.6734]],

         [[0.9755, 1.0455, 0.9930,  ..., 1.2556, 1.2556, 1.2556],
          [0.9755, 1.0280, 0.9930,  ..., 1.2556, 1.2556, 1.2556],
          [0.9580, 1.0105, 0.9930,  ..., 1.2381, 1.2381, 1.2381],
          ...,
          [0.4503, 0.4503, 0.4678,  ..., 0.8179, 0.8179, 0.8179],
          [0.4503, 0.4678, 0.4678,  ..., 0.8179, 0.8179, 0.8179],
          [0.4678, 0.4678, 0.4678,  ..., 0.8179, 0.8179, 0.8179]],

         [[1.1934, 1.2631, 1.2108,  ..., 1.4722, 1.4722, 1.4722],
          [1.1934, 1.2457, 1.2108,  ..., 1.4722, 1.4722, 1.4722],
          [1.1759, 1.2282, 1.2108,  ..., 1

In [9]:
from torchvision import datasets, models, transforms

model_ft = models.resnet18(pretrained=False)


In [10]:
model_ft

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [11]:
import torch.nn as nn
import torch.optim as optim


device = 'cpu'
TARGET_NUM = 10

def get_model(target_num, isPretrained=False):

    model_ft = models.resnet18(pretrained=isPretrained)
    model_ft.fc = nn.Linear(512, target_num)
    model_ft = model_ft.to(device)

model = get_model(TARGET_NUM, isPretrained=False)


In [12]:
optimizer = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

criterion = nn.CrossEntropyLoss()


In [13]:
def train_model(model, criterion, optimizer, num_epochs=5, is_saved=False):
    best_acc = 0.0

    for epoch in range(num_epochs):

        for phase in ['train', 'val']:
            print("{}: phase".format(phase))

            if phase == 'train':
                model.train()
            else:
                model.eval()

            running_loss = 0.0
            running_corrects = 0

            for i,(inputs, labels) in enumerate(image_dataloders[phase]):
                inputs = inputs.to(DEVICE)
                labels = labels.to(DEVICE)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)

                loss = criterion(outputs, labels)
                print('   loaders:{} kaime'.format(i+1),' loss: {}'.format(loss))

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds==labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            print("{} Loss: {:.4f} ACC: {:.4f}".format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                if(is_saved):
                    torch.save(model.state_dict(), './original_model_{}.pth'.format(epoch))
    print("Best val Acc: {:.4f}".format(best_acc))


In [2]:
import pandas as pd
import os

In [3]:
df_test = pd.DataFrame(data=os.listdir('../input/val/'))

In [4]:
df_test

,0
0,459909.jpeg
1,119892.jpeg
2,419909.jpeg
3,339986.jpeg
4,29934.jpeg
...,...
95,59918.jpeg
96,769946.jpeg
97,279912.jpeg
98,269945.jpeg


In [5]:
df_test = df_test.rename(columns={0: 'filename'})

In [7]:
df_test['target'] = 0

In [8]:
df_test.loc[df_test['filename'].str.contains('ok'), 'target'] = 1

In [9]:
df_test.to_csv('dd.csv', index=False)